In [ ]:
def generate_graph(df, hammers, res_lvl, sup_lvl):
    
    import plotly.graph_objects as go

    from plotly.subplots import make_subplots

    INCREASING_COLOR = '#17BECF'
    DECREASING_COLOR = '#7F7F7F'

    df.index = df.Datetime
    
    resistencia = (max(df.Close) - (max(df.Close) - min(df.Close))*res_lvl)
    suporte = (max(df.Close) - (max(df.Close) - min(df.Close))*sup_lvl)
    
    fig = make_subplots(rows=2, cols=1)
    
    fig = fig.add_trace(go.Candlestick(x=df.index,
                                       open=df['Open'],
                                       high=df['High'],
                                       low=df['Low'],
                                       close=df['Close'],
                                    name="candlestick"),
                        row=1, col=1
                       )
    
    fig = fig.add_hline(y=resistencia, line_dash="dot",
              annotation_text="Resistência", 
              annotation_position="bottom right")
    
    fig = fig.add_hline(y=suporte, line_dash="dot",
              annotation_text="Suporte", 
              annotation_position="bottom right")
    
    fig = fig.add_vrect(x0=max(df.loc[df.Close == min(df.Close),:].Datetime), 
                        x1=max(df.loc[df.Close == max(df.Close),:].Datetime), 
                        annotation_text="Área analisada", annotation_position="bottom right",
                        fillcolor="green", opacity=0.25, line_width=0)
    for i in range(0, len(hammers)):
        fig = fig.add_vrect(x0=max(df.loc[df.Datetime == hammers.iloc[i].Datetime - timedelta(minutes=2),:].Datetime),
                            x1=max(df.loc[df.Datetime == hammers.iloc[i].Datetime + timedelta(minutes=1),:].Datetime),
                            annotation_text="Hammer", annotation_position="top left",
                            fillcolor="blue", opacity=0.25, line_width=0)
    fig = fig.update_layout(xaxis_rangeslider_visible=False)
    # volume bar chart

    colors = []

    for i in range(len(df.Close)):
        if i != 0:
            if df.Close[i] > df.Close[i-1]:
                colors.append(INCREASING_COLOR)
            else:
                colors.append(DECREASING_COLOR)
        else:
            colors.append(DECREASING_COLOR)


    # add to graph
    fig.add_trace(
        go.Bar(x=df.index, y=df.Volume,
               marker=dict(color=colors),#
               name='Volume'),
        row=2, col=1

    )
    fig.show()
#(df, res_lvl = 0.236, sup_lvl = 0.764)

In [1]:
from src.logger import logger
from src.directory_utilities import get_json_from_file
from src.database import Database
from src.bittrex import Bittrex
from src.binance import Binance
from src.marketdata import MarketData
from src.trader import Trader
from src.messenger import Messenger
from utils.utils import get_secrets, get_settings, generate_graph
from datetime import datetime, timedelta
from binance.client import Client
import logging
import pandas as pd
import numpy as np

secrets = get_secrets()
settings = get_settings()

global _trader,_db,_bittrex,_botche
_db = Database()
#_bittrex = Bittrex(secrets)
_trader = Trader(secrets, settings, operator = Bittrex)
_messenger = Messenger(secrets, settings)
        
_last_catch = pd.to_datetime(settings.get('last_catch'))
_hora_final = pd.to_datetime(settings.get('hora_final'))
_coin_pair = settings.get('coin_pair')
_unit = settings.get('unit')
_interval = int(settings.get('interval'))
_rsi_buy_threshold = int(settings.get('tradeParameters').get('buy').get('rsiThreshold'))
_rsi_sell_threshold = int(settings.get('tradeParameters').get('sell').get('rsiThreshold'))

#client = Client(api_key, api_secret)
_bittrex = Bittrex(secrets)
_trader = Trader(secrets, settings, operator = MarketData)
_binance = Binance(secrets)._get_client()
_market_data = MarketData(secrets)

In [2]:
tickers = ['PETR4.SA','^BVSP']

In [3]:
%%time
tickers = ['PETR4.SA','^BVSP']
df, crossovers, hammers, suportes, resistencias = _trader.get_historical_prices(tickers[0])

[*********************100%***********************]  1 of 1 completed
Wall time: 20.4 s


In [10]:
df = df.sort_values(['Datetime'])
df[['divergencia_alta']] = 0
df[['divergencia_baixa']] = 0
for i, row in df.iterrows():
    if (i == 0):
        row_m2 = row
    elif (i == 1):
        row_m1 = row
    else:
        df.loc[i,['Tend.Alta (Min d-2 > min(d-1))2']] = (row_m2['Low'] > row_m1['Low'])
        df.loc[i,['Tend.Alta (Min d-1 > min(d))']] = (row_m1['Low'] > row['Low'])
        df.loc[i,['Tend.Alta (Hist d-2 < histd-1)']] = (row_m2['MACD_Histogram'] < row_m1['MACD_Histogram'])
        df.loc[i,['Tend.Alta (Hist d-1 < histd)2']] = (row_m1['MACD_Histogram'] < row['MACD_Histogram'])
        df.loc[i,['MACD < 0']] = (row['MACD'] < 0)
        df.loc[i,['Tend.Baixa(Máx d-1 & d-2)']] = (row_m2['High'] < row_m1['High'])
        df.loc[i,['Tend.Baixa(Máx d & d-1)']] = (row_m1['High'] < row['High'])
        df.loc[i,['Tend.Baixa (Hist d & d-1)']] = (row_m1['MACD_Histogram'] > row['MACD_Histogram'])
        df.loc[i,['Tend.Baixa (Hist d-2 & d-1)']] = (row_m2['MACD_Histogram'] > row_m1['MACD_Histogram'])
        row_m1 = row
        row_m2 = row_m1

In [ ]:
row_m2 = df.loc[df.Datetime == pd.to_datetime('2018-12-26 21:00:00'),
       ['Datetime','Low','MACD_Histogram', 'Support']]
row_m1 =  df.loc[df.Datetime == pd.to_datetime('2018-12-26 21:00:00'),
       ['Datetime','Low','MACD_Histogram', 'Support']]
row =  df.loc[df.Datetime == pd.to_datetime('2018-12-26 21:00:00'),
       ['Datetime','Low','MACD_Histogram', 'Support']]

In [22]:
df.loc[df.Datetime <= pd.to_datetime('2018-12-26 21:00:00'),
       ['Datetime','Low','MACD_Histogram','Volume','Tend.Alta (Min d-2 > min(d-1))2', 'Tend.Alta (Min d-1 > min(d))','Tend.Alta (Hist d-2 < histd-1)','Tend.Alta (Hist d-1 < histd)2','MACD < 0','Support']]

,Datetime,Low,MACD_Histogram,Volume,Tend.Alta (Min d-2 > min(d-1))2,Tend.Alta (Min d-1 > min(d)),Tend.Alta (Hist d-2 < histd-1),Tend.Alta (Hist d-1 < histd)2,MACD < 0,Support
0,2016-01-25 21:00:00,4.120000,0.000000,71933300.0,NaN,NaN,NaN,NaN,NaN,1
1,2016-01-26 21:00:00,4.160000,0.021192,79437900.0,NaN,NaN,NaN,NaN,NaN,0
2,2016-01-27 21:00:00,4.350000,0.035034,131960900.0,False,False,True,True,True,0
3,2016-01-28 21:00:00,4.520000,0.055492,93013900.0,False,False,False,True,True,0
4,2016-01-31 21:00:00,4.660000,0.058277,51910300.0,False,False,False,True,True,0
...,...,...,...,...,...,...,...,...,...,...
727,2018-12-18 21:00:00,22.059999,-0.258378,171157500.0,False,False,False,True,False,0
728,2018-12-19 21:00:00,21.350000,-0.278114,141767000.0,False,True,False,False,False,0
729,2018-12-20 21:00:00,21.250000,-0.268085,90384300.0,False,True,False,True,False,0
730,2018-12-25 21:00:00,20.420000,-0.192199,77104200.0,False,True,False,True,False,1


In [ ]:
for i in range(2, len(df)):
    if ((df.loc[i-2, ['Low']][0]) > ((df.loc[i-1, ['Low']][0])) > (df.loc[i, ['Low']][0])):
        if ((df.loc[i-2, ['MACD_Histogram']][0]) < ((df.loc[i-1, ['MACD_Histogram']][0])) < (df.loc[i, ['MACD_Histogram']][0])):
            if (df.loc[i, ['Support']][0] == 1 and df.loc[i, ['MACD']][0]<0):
                df.loc[i,['divergencia_alta']] = 1
    if ((df.loc[i-2, ['High']][0]) < ((df.loc[i-1, ['High']][0])) < (df.loc[i, ['High']][0])):
        if ((df.loc[i-2, ['MACD_Histogram']][0]) > ((df.loc[i-1, ['MACD_Histogram']][0])) > (df.loc[i, ['MACD_Histogram']][0])):
            df.loc[i,['divergencia_baixa']] = 1

In [15]:
get_tendencia_baixa_divergencia(get_tendencia_alta_divergencia(df)).to_csv('df.csv', sep=';', decimal=',')

-------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------

In [6]:
df = _bittrex.get_historical_data(market = "USD-BTC", period=14*3, unit=_unit)
df[1:3]

[{'O': 62734.09,
  'H': 62738.58,
  'L': 62734.09,
  'C': 62734.09,
  'V': 0.01151296,
  'T': '2021-04-13T13:44:00',
  'BV': 722.27497524},
 {'O': 62749.47,
  'H': 62805.199,
  'L': 62734.15,
  'C': 62804.89,
  'V': 0.20228792,
  'T': '2021-04-13T13:45:00',
  'BV': 12701.77220153}]

In [3]:
df2 = Binance(secrets).get_historical_data(coin_pair = _coin_pair, period = 14*3, unit=_unit)
df2[1:3]

[{'O': 561.8815,
  'BV': 3961368.5245323,
  'C': 562.5213,
  'H': 563.6647,
  'L': 561.2718,
  'T': Timestamp('2021-04-13 13:04:59.999000'),
  'V': 7040.772},
 {'O': 562.5213,
  'BV': 3340361.5419088,
  'C': 565.102,
  'H': 565.2701,
  'L': 562.5072,
  'T': Timestamp('2021-04-13 13:05:59.999000'),
  'V': 5920.95}]

In [4]:
_trader.calculate_rsi(_coin_pair, period=14, unit=_unit)

55.20186006031749